In [3]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Loading data into pandas and do exploratoin

In [4]:
# load data
test_data = pd.read_csv("/Users/yijia/Documents/current_work/BigData/Assign/Final/data/test.csv")
train_data = pd.read_csv('/Users/yijia/Documents/current_work/BigData/Assign/Final/data/train.csv')
attributes = pd.read_csv("/Users/yijia/Documents/current_work/BigData/Assign/Final/data/attributes.csv")
product_description = pd.read_csv("/Users/yijia/Documents/current_work/BigData/Assign/Final/data/product_descriptions.csv")

### Problem statement:
Giving training data contains id, product_uid, product title, and search term, given the relevance as label. We can also used the attributes and product description in the training information, by joining with product_uid. 
Our task is to predict the relevance for data in test set given id and product_uid. 

In [5]:
train_data.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [91]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74067 entries, 0 to 74066
Data columns (total 5 columns):
id               74067 non-null int64
product_uid      74067 non-null int64
product_title    74067 non-null object
search_term      74067 non-null object
relevance        74067 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.8+ MB


In [6]:
test_data.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [7]:
attributes.head()

,product_uid,name,value
0,100001.0,Bullet01,Versatile connector for various 90° connection...
1,100001.0,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001.0,Bullet03,Help ensure joints are consistently straight a...
3,100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001.0,Bullet05,Made from 12-Gauge steel


In [92]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952634 entries, 0 to 2044802
Data columns (total 3 columns):
product_uid    float64
name           object
value          object
dtypes: float64(1), object(2)
memory usage: 59.6+ MB


In [ ]:
product_description.head()

### Corpus exploration:
For this task, corpus can be seen as the combination of product description which conatins product_uid and product description for this unique product_uid. And the attributes, which contains map of product_uid and name and value. 
To explore the corpus, we can calculate each word's term frequency as well as the invert document frequency in the product description file. Here we can consider each single description as a document in corpus. Also we can seek into the attribut of product and see what relationship the attribut has to the query. 

In [130]:
# terms in product description
terms = dict()
uids = product_description['product_uid']
docs = product_description['product_description']
for i in range(0, len(product_description)):
    terms[uids[i]] = re.split("\W+", docs[i])


#### Calculate Tf and idf of term, score can be represent as tf*idf

In [132]:
# explore prodcut description
# calculate tf and idf for each term in a single product's description.
import re

def calculateTf(uid, t):
#     cur_doc = product_description[product_description['product_uid'] == uid]['product_description'][0]
#     terms = set(re.split("\W+", cur_doc))
    term = terms[uid]
#     tf = dict()
#     for t in term:
#         tf[t] = cur_doc.count(t)
    tf = term.count(t)
    return tf

def calculateIdf(uid, t):
#     cur_doc = product_description[product_description['product_uid'] == uid]['product_description'][0]
#     terms = set(re.split("\W+", cur_doc))
    term = terms[uid]
#     idf = dict()
    N = len(product_description)
#     for t in terms:
#         idf[t] = np.log(N/(getDf(t) + 1))
    idf = np.log(N/(getDf(t) + 1))
    return idf

def getDf(term):
    cnt = 0
    for t in terms.values():
        if term in t:
            cnt += 1
    return cnt

In [71]:
# scores can be represent as tf*idf
def getScore(uid):
    return calculateIdf(uid)*calculateTf(uid)

#### Explore attributes, using information about brands, material, etc

In [75]:
# explore attributes
attributes = attributes[attributes['value'] != 'No']
brands = attributes[attributes['name'] == "MFG Brand Name"]
material = attributes[attributes['name'] == "Material"]

In [76]:
material.head()

,product_uid,name,value
8,100001.0,Material,Galvanized Steel
67,100003.0,Material,Composite
202,100007.0,Material,Plastic
255,100009.0,Material,Medium Density Fiberboard (MDF)
282,100010.0,Material,Steel


In [77]:
brands.head()

,product_uid,name,value
9,100001.0,MFG Brand Name,Simpson Strong-Tie
37,100002.0,MFG Brand Name,BEHR Premium Textured DeckOver
69,100003.0,MFG Brand Name,STERLING
93,100004.0,MFG Brand Name,Grape Solar
122,100005.0,MFG Brand Name,Delta


#### Finding: 
the attributes for each product_uid may be different. For example, some products do not have the informantion about their brand or material. But all the products have product description which we can retrive some information from. Also, the title can also be used to make prediction for relevance. We can see those with related title will more likely to be relevanted. So basically using the decription's tf-idf score, brand, title, material as features and decided whether it is relevant to the query. When processing the description, the data set is very large, we only need to calculate terms appears in search query, but it still cost time to calculate the tf and idf. 

#### Approach to solve the problem:
After deciding which feature to use and feature engineering, we can decide which model to use to make the prediction. Basically machine learning algorithms will be used for prediction. I think for this task, the result is a number in the range of [0, 3], so in this way we can ues regression model as baseline model. For example, we may quantify the feature of the train and test data, use integer to represent str features, and then apply linear regression model to make the prediction. Also, after base line model, I think we can use some model like random forest since RF can handle both classification and regression, and both discrete and continus features work well. So in this way, I may try RF model on the training set and test to see how the model works. Other models like CNN may also be used. And since there are several terms in each query,I pretend to combine them by "OR", "AND", two different operators to make the query, then compare the result's performance. And since we have different field like title and description and attributes, we can try to find different weight for the field and use weight operator like "WAND" to get the relevance.

In [98]:
# Feature engineering
# merge train and test with description
train = train_data.merge(product_description, on = "product_uid", how = 'left')
test = test_data.merge(product_description, on = "product_uid", how = 'left')

In [99]:
train.head()

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...


In [100]:
test.head()

,id,product_uid,product_title,search_term,product_description
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ..."
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ..."
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"Not only do angles make joints stronger, they ..."
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties,"Not only do angles make joints stronger, they ..."
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668,"Not only do angles make joints stronger, they ..."


In [101]:
# merge train and test with brand and material
train = train.merge(brands, on = "product_uid", how = 'left')
test = test.merge(brands, on = "product_uid", how = 'left')
train = train.merge(material,on = "product_uid", how = 'left')
test = test.merge(material, on = "product_uid", how = 'left')

In [102]:
train.head()

,id,product_uid,product_title,search_term,relevance,product_description,name_x,value_x,name_y,value_y
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,MFG Brand Name,BEHR Premium Textured DeckOver,NaN,NaN
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,MFG Brand Name,Delta,NaN,NaN
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,MFG Brand Name,Delta,NaN,NaN


In [104]:
test.head()

,id,product_uid,product_title,search_term,product_description,name_x,value_x,name_y,value_y
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel


### Calculate AND and OR operator's result of each term and save

In [167]:
# calculate AND score, OR score and add new column to train
search_terms = []
for st in train['search_term']:
    search_terms.append(re.split("\W+", st))
AND_score = []
OR_score = []
idx = 0
for ts in search_terms:
    cur_uid = train['product_uid'][idx]
    cur_and = 1 # and , multiply all terms' score
    cur_or = 0 # or, find max one 
    for t in ts:
        cur_tf = calculateTf(cur_uid, t)
        cur_idf = calculateIdf(cur_uid, t)
        cur_and = cur_and * cur_tf*cur_idf
        cur_or = max(cur_or, cur_tf*cur_idf)
    AND_score.append(cur_and)
    OR_score.append(cur_or)
    idx += 1
train['AND_score'] = AND_score
train['OR_score'] = OR_score

In [ ]:
# calculate AND score, OR score and add new column to test
search_terms_t = []
for st in test['search_term']:
    search_terms_t.append(re.split("\W+", st))
AND_score_t = []
OR_score_t = []
idx = 0
for ts in search_terms_t:
    cur_uid = test['product_uid'][idx]
    cur_and = 1 # and , multiply all terms' score
    cur_or = 0 # or, find max one 
    for t in ts:
        cur_tf = calculateTf(cur_uid, t)
        cur_idf = calculateIdf(cur_uid, t)
        cur_and = cur_and * cur_tf*cur_idf
        cur_or = max(cur_or, cur_tf*cur_idf)
    AND_score_t.append(cur_and)
    OR_score_t.append(cur_or)
    idx += 1
test['AND_score'] = AND_score_t
test['OR_score'] = OR_score_t

In [154]:
# process with product title, the percent of terms that appear in title 
train_title = []
test_title = []
idx = 0
for ts in train['search_term']:
    cur_terms = set(re.split("\W+", ts))
    cur_title = set(re.split("\W+", train['product_title'][idx]))
    train_title.append(float(len(cur_terms.intersection(cur_title)))/len(cur_title))
    idx += 1
train['title_contain'] = train_title

In [155]:
idx = 0
for ts in test['search_term']:
    cur_terms = set(re.split("\W+", ts))
    cur_title = set(re.split("\W+", test['product_title'][idx]))
    test_title.append(float(len(cur_terms.intersection(cur_title)))/len(cur_title))
    idx += 1
test['title_contain'] = test_title

In [156]:
train.head()

,id,product_uid,product_title,search_term,relevance,product_description,name_x,value_x,name_y,value_y,score,title_contain
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0,0.0
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0,0.0
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,MFG Brand Name,BEHR Premium Textured DeckOver,NaN,NaN,0,0.0
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,MFG Brand Name,Delta,NaN,NaN,0,0.0
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,MFG Brand Name,Delta,NaN,NaN,0,0.0


In [157]:
test.head()

,id,product_uid,product_title,search_term,product_description,name_x,value_x,name_y,value_y,title_contain
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0.0
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0.0
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0.0
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0.0
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0.0


In [162]:
train.head()

,id,product_uid,product_title,search_term,relevance,product_description,name_x,value_x,name_y,value_y,score,title_contain,AND_score,OR_score
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0,0.0,0.0,0.000000
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",MFG Brand Name,Simpson Strong-Tie,Material,Galvanized Steel,0,0.0,0.0,0.000000
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,MFG Brand Name,BEHR Premium Textured DeckOver,NaN,NaN,0,0.0,0.0,3.713572
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,MFG Brand Name,Delta,NaN,NaN,0,0.0,0.0,0.000000
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,MFG Brand Name,Delta,NaN,NaN,0,0.0,0.0,2.397895


#### Findings:
For those with high relevance, OR operator have a higher score than those with low relevance, that means this score can be used as features for prediction. Since AND operator's result requires all terms appears, it is usuallly 0, which means this operator is not a good choice. However, for some relevance which is not that bad, the OR score can also be zero, that means we may use other features and tune their weight in order to get a better result. Using tf-idf socre based on description only will not be a good choice. 

### Next steps:
Baseline model: linear regression - using spark mllib

Other ML model: Random forest - regression and classification, using spark mllib